# Using and analyzing foldlabel files

# Imports and definitions

In [35]:
import numpy as np
from soma import aims

In [36]:
import anatomist.notebook as ana
a = ana.Anatomist()
print(a.headless_info.__dict__)

{'xvfb': <subprocess.Popen object at 0x7f35a8bbb048>, 'original_display': ':1', 'display': 3, 'glx': 2, 'virtualgl': None, 'headless': True, 'mesa': False, 'qtapp': None}


In [37]:
src_dir="/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rlabels"

In [38]:
skel_dir="/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rcrops"

In [39]:
subject="100206"

In [40]:
# Defines the function to plot the volume in anatomist
def plot_sagittal(vol):
    global a
    print(vol.header())
    a_vol = a.toAObject(vol)
    sagittal = a.createWindow('Sagittal')
    sagittal.addObjects(a_vol)
    return sagittal

In [41]:
foldlabel_file = f"{src_dir}/{subject}_cropped_foldlabel.nii.gz"

In [42]:
skeleton_file = f"{skel_dir}/{subject}_cropped_skeleton.nii.gz"

# Checks and plots

In [43]:
vol = aims.read(foldlabel_file)

In [44]:
vol_skel = aims.read(skeleton_file)

In [45]:
win1 = plot_sagittal(vol)

{ 'volume_dimension' : [ 17, 33, 36, 1 ], 'sizeX' : 17, 'sizeY' : 33, 'sizeZ' : 36, 'sizeT' : 1, 'disk_data_type' : 'S16', 'bits_allocated' : 16, 'data_type' : 'S16', 'scale_factor_applied' : 0, 'possible_data_types' : [ 'S16', 'FLOAT', 'DOUBLE' ], 'cal_min' : 0, 'cal_max' : 0, 'freq_dim' : 0, 'phase_dim' : 0, 'slice_dim' : 0, 'slice_code' : 0, 'slice_start' : 0, 'slice_end' : 0, 'slice_duration' : 0, 'storage_to_memory' : [ -1, 0, 0, 16, 0, -1, 0, 32, 0, 0, -1, 35, 0, 0, 0, 1 ], 'voxel_size' : [ 2, 2, 2 ], 'tr' : 1, 'referentials' : [ 'Scanner-based anatomical coordinates' ], 'transformations' : [ [ -1, 0, 0, 17, 0, -1, 0, 31, 0, 0, -1, 34, 0, 0, 0, 1 ] ], 'toffset' : 0, 'xyz_units' : 0, 'time_units' : 0, 'descrip' : '', 'aux_file' : '', 'nifti_type' : 1, 'object_type' : 'Volume', 'file_type' : 'NIFTI-1' }


AnatomistInteractiveWidget(height=308, layout=Layout(height='auto', width='auto'), width=424)

# General computation

In [46]:
arr_foldlabel = np.asarray(vol)

In [47]:
arr_skel = np.asarray(vol_skel)

In [48]:
arr_foldlabel.shape

(17, 33, 36, 1)

In [49]:
branches, nb_per_branch = np.unique(arr_foldlabel, return_counts=True)
print(branches)
print(nb_per_branch)
print(f"nb of branches = {branches.shape[0]}")
print(type(nb_per_branch))

[   0 1005 1024 1048 1087 1098 1255 1259 1276 1311 1346 1360 1367 1375
 1393 1405 1408 2005 2024 2034 2083 2087 2098 2255 2259 2276 2311 2346
 2360 2367 2375 2393 2405 2408 3156 3165 3500 3606 3619 3949 3951 4002
 4006 4011 4017 4019 4027 4040 4047 4048 4419 4842 4856]
[19394    23     6     6     5    31     3   198     4     1    10     2
     4    39     5    21    69     6     5     2     1     9    19     1
    63     4     4     7     2     4    13     7    17    19     6     3
     1     4    17     3     2    67     6     2    16     2    14    19
    21     3     1     3     2]
nb of branches = 53
<class 'numpy.ndarray'>


In [50]:
nb_per_branch[nb_per_branch>6].sum()-nb_per_branch[0]

699

In [51]:
np.unique(arr_skel, return_counts=True)

(array([  0,  30,  60, 110], dtype=int16), array([19505,   192,   494,     5]))

In [52]:
arr_skel.shape

(17, 33, 36, 1)

In [53]:
# Counts non-zero values
(arr_foldlabel != 0).sum()

802

In [54]:
(arr_skel != 0).sum()

691

In [55]:
diff = arr_skel[arr == 0]

In [56]:
np.count_nonzero(diff)

0

In [57]:
diff_inverse = arr_foldlabel[arr_skel == 0]
np.count_nonzero(diff_inverse)

111

In [58]:
nb_per_branch

array([19394,    23,     6,     6,     5,    31,     3,   198,     4,
           1,    10,     2,     4,    39,     5,    21,    69,     6,
           5,     2,     1,     9,    19,     1,    63,     4,     4,
           7,     2,     4,    13,     7,    17,    19,     6,     3,
           1,     4,    17,     3,     2,    67,     6,     2,    16,
           2,    14,    19,    21,     3,     1,     3,     2])

# Algorithm to remove branches

In [59]:
branches.size

53

In [60]:
np.random.randint(0,52)+1

50

In [61]:
def select_one_random_branch(arr_foldlabel):
    """It selects randomly one of the branch
    
    The branch is characterized by a number.
    This number is present on several pixels in the array"""

    branches = np.unique(arr_foldlabel)
    nb_branches = branches.size
    # 0 is not a branch
    selected_branch = np.random.randint(0,nb_branches-1)+1
    return branches[selected_branch]

In [126]:
def remove_branch(arr_foldlabel, arr_skel, selected_branch):
    """It masks the selected branch in arr_skel
    """
    # print((arr_foldlabel > 0).sum())
    mask = ( (arr_foldlabel != 0) & (arr_foldlabel != selected_branch))  
    mask = mask.astype(int)
    # print(mask.sum())
    return arr_skel * mask

In [63]:
select_one_random_branch(arr_foldlabel)

1360

In [105]:
selected_branch=1005
arr_skel_without_branch = remove_branch(arr_foldlabel, arr_skel, selected_branch)

In [65]:
# Expected 23
np.count_nonzero(arr_skel)-np.count_nonzero(arr_skel_without_branch)

23

In [66]:
selected_branch=1311
arr_skel_without_branch = mask_branch(arr_foldlabel, arr_skel, selected_branch)
# Expected 1
np.count_nonzero(arr_skel)-np.count_nonzero(arr_skel_without_branch)

802
801


1

We want to check which branches are not represented at all in skeleton.
Indeed, there are 111 pixels in foldlabel that have a 0 value correspondence
in skeleton

In [68]:
branches = np.unique(arr_foldlabel)
print(branches)

[   0 1005 1024 1048 1087 1098 1255 1259 1276 1311 1346 1360 1367 1375
 1393 1405 1408 2005 2024 2034 2083 2087 2098 2255 2259 2276 2311 2346
 2360 2367 2375 2393 2405 2408 3156 3165 3500 3606 3619 3949 3951 4002
 4006 4011 4017 4019 4027 4040 4047 4048 4419 4842 4856]


In [103]:
indexes = np.arange(branches.size-1)+1
np.random.shuffle(indexes)
indexes

array([ 7, 43, 37, 22, 40, 15,  5,  9, 12, 34, 20, 48, 28,  1, 25, 21, 29,
       27,  2, 10, 13,  4, 51, 26, 41, 30, 36,  3, 31, 50, 52, 38, 11, 39,
       23,  6, 45, 18, 42, 35, 33,  8, 14, 16, 47, 32, 19, 24, 46, 17, 44,
       49])

In [97]:
histo_foldlabel = np.unique(arr_foldlabel, return_counts=True)
histo_foldlabel = np.stack((histo_foldlabel),axis=-1)
for branch in branches:
    if branch != 0:
        arr_skel_without_branch = mask_branch(arr_foldlabel, arr_skel, branch)
        diff = np.count_nonzero(arr_skel)-np.count_nonzero(arr_skel_without_branch)
        if diff == 0:
            index = np.argwhere(histo_foldlabel[:,0]==branch)
            print(histo_foldlabel[index])

[[[3951    2]]]
[[[4011    2]]]
[[[4019    2]]]
[[[4419    1]]]


We note and conclude that very few branches have no correspondence with skeleton

# Full program to remove branches up to the point for which a certain percentage of pixels have been removed

In [127]:
def remove_branches_up_to_percent(arr_foldlabel, arr_skel, percentage):
    """Removes from arr_skel random branches up percentage of pixels
    """
    branches = np.unique(arr_foldlabel)
    # We take as index branches indexes that are not 0
    indexes = np.arange(branches.size-1) + 1
    # We take random branches
    np.random.shuffle(indexes)
    arr_skel_without_branches = arr_skel
    total_pixels = (arr_skel !=0 ).sum()
    total_pixels_after=total_pixels
    for index in indexes:
        if total_pixels_after <= total_pixels*(100-percentage)/100:
            break
        arr_skel_without_branches = \
            remove_branch(arr_foldlabel,
                          arr_skel_without_branches,
                          branches[index])
        total_pixels_after = (arr_skel_without_branches != 0).sum()
    print(f"total_pixels_after = {total_pixels_after}")
    print(f"% removed pixels = {(total_pixels-total_pixels_after)/total_pixels*100}")
    return arr_skel_without_branches

In [135]:
arr_skel_without_branches = remove_branches_up_to_percent(arr_foldlabel, arr_skel, 10)

total_pixels_after = 581
% removed pixels = 15.918958031837915
